In [24]:
#Running the core utilities notebook with all validation and loading functions
%run transform_utils.ipynb

In [25]:
#load data
transaction_df, counts = read_transactions_csv(transactions_bronze_path)
len(transaction_df)

85347

In [26]:
# validate if the sum of the rows in individual files is equal to the number of rows in the final df 
assert sum(counts) == len(transaction_df)
print(sum(counts),' = ',len(transaction_df))

85347  =  85347


In [27]:
#check date type - assertiton can be added
transaction_df.dtypes

location_id               object
date              datetime64[ns]
transaction_id            object
profit                   float64
dtype: object

In [58]:
# copy the date column
date_df = transaction_df[['date']].copy()
print(len(date_df))

#drop all duplicate dates
date_df.drop_duplicates( "date" , keep='first', inplace= True)
print(len(date_df))

#set a new column holiday as false
date_df['holiday'] = False
date_df

85347
827


,date,holiday
0,2019-01-02,False
21,2019-01-03,False
41,2019-01-04,False
55,2019-01-05,False
70,2019-01-06,False
...,...,...
81154,2021-03-05,False
81165,2021-03-06,False
81181,2021-03-07,False
81191,2021-03-08,False


In [59]:
#load holiday
holiday_df = pd.read_csv('../Submission/bronze_data/holiday_data/holiday_data.csv',dtype={'holiday':bool}, parse_dates= ['date'])
holiday_df

,date,holiday
0,2019-01-01,True
1,2019-05-27,True
2,2019-07-04,True
3,2019-09-02,True
4,2019-11-28,True
5,2019-12-25,True
6,2020-01-01,True
7,2020-05-25,True
8,2020-07-04,True
9,2020-09-07,True


In [68]:
#join both dataframes
joined_df = pd.merge(date_df, holiday_df, on='date', how='outer')
joined_df['holiday_x'] = joined_df['holiday_x'].fillna(False)
joined_df['holiday_y'] = joined_df['holiday_y'].fillna(False)
joined_df.head()


,date,holiday_x,holiday_y
0,2019-01-02,False,False
1,2019-01-03,False,False
2,2019-01-04,False,False
3,2019-01-05,False,False
4,2019-01-06,False,False


In [69]:
# make a new column holiday, such that overlapping days that are holidays will be correctly labelled as holidays
joined_df['holiday'] = joined_df['holiday_x'] | joined_df['holiday_y']
#drop the columns not needed
joined_df.drop(columns=['holiday_x', 'holiday_y'],inplace=True)
joined_df.sort_values(by='date',inplace=True)
joined_df

,date,holiday
827,2019-01-01,True
0,2019-01-02,False
1,2019-01-03,False
2,2019-01-04,False
3,2019-01-05,False
...,...,...
752,2022-09-07,False
753,2022-09-08,False
754,2022-09-09,False
755,2022-09-10,False


In [70]:
# derive day,monthm year, day_of_week from the existing columns
joined_df['day'] = joined_df['date'].dt.day
joined_df['month'] = joined_df['date'].dt.month
joined_df['year'] = joined_df['date'].dt.year
joined_df['day_of_week'] = joined_df['date'].dt.day_of_week
joined_df.head()

,date,holiday,day,month,year,day_of_week
827,2019-01-01,True,1,1,2019,1
0,2019-01-02,False,2,1,2019,2
1,2019-01-03,False,3,1,2019,3
2,2019-01-04,False,4,1,2019,4
3,2019-01-05,False,5,1,2019,5


In [71]:
# save file to silver path
file_name = 'date_silver_data.csv'
write_to_path_silver(joined_df,silver_date_path,file_name)

DataFrame saved to ../Submission/silver_data/silver_date/date_silver_data.csv


In [72]:
# open db connection
conn = sqlite3.connect('tubing.db')
cursor = conn.cursor()

In [73]:
#add rows to db - I would prefer to uspert or overwrite using pyspark/sparksql - but since sqlLite is limited I am using write
joined_df.to_sql('temp_date', conn, if_exists='replace', index=False)

849

In [75]:
# test if rows are in the table
query = """
SELECT * FROM date LIMIT 10
"""
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Print the results
if results:
    print("Fetched rows:")
    for row in results:
        print(row)
else:
    print("No rows fetched. The table might be empty.")

Fetched rows:
('2019-01-01T00:00:00', 1, 1, 1, 2019, 1)
('2019-01-02T00:00:00', 2, 2, 1, 2019, 0)
('2019-01-03T00:00:00', 3, 3, 1, 2019, 0)
('2019-01-04T00:00:00', 4, 4, 1, 2019, 0)
('2019-01-05T00:00:00', 5, 5, 1, 2019, 0)
('2019-01-06T00:00:00', 6, 6, 1, 2019, 0)
('2019-01-07T00:00:00', 7, 0, 1, 2019, 0)
('2019-01-08T00:00:00', 8, 1, 1, 2019, 0)
('2019-01-09T00:00:00', 9, 2, 1, 2019, 0)
('2019-01-10T00:00:00', 10, 3, 1, 2019, 0)


In [76]:
#close connection
conn.close()